In [377]:
from random import randrange
import math

In [378]:
def to_hex_64chunks(t):
    res = []
    while t > 0:
        res.append(t % 2^64)
        t = t >> 64
    return res

def hex_64chunks_to_string(chunks):
    res = [hex(chunk).upper().replace('X', 'x') for chunk in chunks]
    res_string = '{' + ', '.join(res) + '}'
    return res_string

def from_hex_64chunks(chunks):
    res = 0
    for chunk in chunks[::-1]:
        res = res << 64
        res += chunk
    return res

def get_prime_details(pow_2, pow_3, cofactor, bits):
    res = {}

    res["pow_2"] = pow_2
    res["pow_3"] = pow_3
    res["cofactor"] = cofactor

    p = 2^pow_2 * 3^pow_3 * cofactor - 1
    res["p"] = p
    res["p_bits"] = math.ceil(math.log2(p))

    R = 1 << bits
    res["R"] = R
    res["R_bits"] = bits

    p1 = int(1 + p)
    res["p1"] = p1
        
    px2 = p * 2
    res["px2"] = px2
    
    R = IntegerModRing(2^(bits))
    temp = R(p)
    temp = temp^-1
    temp = -1 * temp
    pinv_R = int(temp)
    res["pinv_R"] = pinv_R # Modulo AND additive inverse: p * pinv_R === -1 mod R

    P = IntegerModRing(p)
    R2 = 2^(bits * 2)
    temp = P(R2)
    R2_p = int(temp)
    res["R2_p"] = R2_p

    R = 2^(bits)
    temp = P(R)
    R_p = int(temp)
    res["R_p"] = R_p

    return res

def print_prime_details(prime_details):
    for key in prime_details.keys():
        if "bits" in key:
            print(key, ':', prime_details[key])
            continue
        print(key, ':', hex_64chunks_to_string(to_hex_64chunks(prime_details[key])))

def print_prime_details2(pow_2, pow_3, cofactor, bits):
    print('p = 2^{} * 3^{} * {} - 1'.format(pow_2, pow_3, cofactor))
    p = 2^pow_2 * 3^pow_3 * cofactor - 1
    
    print('p bits = {}'.format(math.log(p, 2)))

    print('prime')
    temp = p
    print(hex_64chunks_to_string(to_hex_64chunks(temp)))

    print('primeplus1')
    temp = int(1 + p)
    print(hex_64chunks_to_string(to_hex_64chunks(temp)))
        
    print('prime*2')
    temp = p * 2
    print(hex_64chunks_to_string(to_hex_64chunks(temp)))
    
    print('Mont prime_inv mod R')
    R = IntegerModRing(2^(bits))
    temp = R(p)
    temp = temp^-1
    temp = -1 * temp
    temp = int(temp)
    print(hex_64chunks_to_string(to_hex_64chunks(temp)))

    P = IntegerModRing(p)
    R2 = 2^(bits * 2)
    print('Mont R^2 mod prime')
    temp = P(R2)
    temp = int(temp)
    print(hex_64chunks_to_string(to_hex_64chunks(temp)))

    print('Mont R mod prime')
    R = 2^(bits)
    temp = P(R)
    temp = int(temp)
    print(hex_64chunks_to_string(to_hex_64chunks(temp)))

def check_supersingular(p):
    F2.<a> = GF(p^2, modulus=x^2 + 1) # Note; not using GF(p^2) because of a limitation in Sage
    A = F2(6)
    B = F2(1)
    C = F2(1)
    S = B^1 * (1 - A^2/3)
    T = B^3 * A / 3 * (2*A^2/9 - 1)
    E = EllipticCurve(F2, [S, T]) # v^2 = u^3 + (B^2(1 - A^2/3))u + B^3 * A / 3(2A^2/9 - 1)
    return E.is_supersingular()

def to_mont(prime_details, elem):
    return (elem << prime_details["R_bits"]) % prime_details["p"]

def mont_red(prime_details, m_elem):
    m = ((m_elem & ((1 << prime_details["R_bits"]) - 1)) * prime_details["pinv_R"]) & ((1 << prime_details["R_bits"]) - 1)
    t = (m_elem + m * prime_details["p"]) >> prime_details["R_bits"]
    if t >= prime_details["p"]:
        print("@@@@@@@")
        return t - prime_details["p"]
    else:
        return t

def compute_y(E, x):
    R.<y> = PolynomialRing(E.base_ring())
    a = E.a_invariants()
    L = (y^2 + (a[0]*x + a[2])*y - (x^3 + a[1]*x^2 + a[3]*x + a[4])).roots()
    return [l[0] for l in L]

def mont_xDBL(A, x):
    return ((x^2 - 1)^2)/(4*x*(x^2 + A*x + 1))

def mont_2isog(A, B, kernel_x, ring):
    T.<x> = PolynomialRing(ring)
    # Rational x-coordinate map for isogeny
    pol = x*(x*kernel_x - 1)/(x - kernel_x)
    new_A = 2*(1 - 2*kernel_x^2)
    new_B = kernel_x*B
    
    def isog(input_x):
        return pol(x=input_x)
    return isog, new_A, new_B


def gen_2ea_generators(E, prime_details, mont_gen=False):
    ea = prime_details["pow_2"]
    eb = prime_details["pow_3"]
    while(True):
        P = (3**eb) * E.random_point()
        # Check if order of P is correct,
        # and if we are generating points from a Montgomery curve E, that 2^ea(P) is not (0 : 0 : 1)
        if P.order() == (2**ea) and (not mont_gen or 2**(ea - 1) * P != E(0, 0)):
            while(True):
                Q = (3**eb) * E.random_point()
                # Check if order of Q is correct
                if Q.order() == (2**ea):
                    # Now check if P and Q are linearly independent
                    if (2**(ea - 1) * P != 2**(ea - 1) * Q):
                        # And if we are generating points from a Montgomery curve E, that 2^ea(Q) is (0 : 0 : 1)
                        if (not mont_gen) or  2**(ea - 1) * Q == E(0, 0):
                            return P, Q
        
def get_isog(E, R, l):
    """ Calculate the (codomain of the) isogeny using SageMath's implementation.

    Keyword arguments:
    E -- the domain elliptic curve
    R -- the point in E with order l^e, which generates the kernel of isogeny
    l -- the exponent base of order of R
    """
    ord = int(math.log(R.order(), l))
    RA = R
    EA = E
    # Pushing kernel point through each consecutive isogeny
    # TODO Use strategy 
    for i in range(ord - 1, -1, -1):
        next_kernel_point = (l**i) * RA
        isog = EA.isogeny(next_kernel_point)
        EA = isog.codomain()
        if (i == 0):
            # Skip the last isogeny push
            break
        RA = isog(RA)
    return EA

# Calculate the (codomain of the) isogeny using Montgomery formulas
def get_mont_2_isog(A, B, R0, R1, ord, ring):
    """ Calculate the (Montgomery form of the codomain of the) isogeny using Montgomery formulas

    Keyword arguments:
    A -- the A coefficient of domain elliptic curve, in Montgomery form By^2 = x^3 + Ax^2 + x
    B -- the B coefficient of domain elliptic curve, in Montgomery form By^2 = x^3 + Ax^2 + x
    R0 -- the x-coordinate of the point in E with order 2^e, which generates the kernel of isogeny
    R1 -- the y-coordinate of the point in E with order 2^e, which generates the kernel of isogeny
    ord -- the order of the point (R0, R1) in E
    ring -- the ring under which elliptic curve is defined
    """
    RAx = R0
    T.<x> = PolynomialRing(ring)
    
    for i in range(ord - 1, -1, -1):
        # Apply Montgomery xDBL formula to get the x-coordinate of next 2-order kernel point
        # https://eprint.iacr.org/2017/212.pdf, 3.1 eq 8
        next_kernel_x = RAx
        for j in range(i):
            next_kernel_x = mont_xDBL(A, next_kernel_x)

        isog, A, B = mont_2isog(A, B, next_kernel_x, ring)
        # Push initial kernel generator point x-coordinate through isogeny
        if (i == 0):
            # Don't need to push through the last isogeny
            break
        RAx = isog(RAx)
    return {
        "A": A,
        "B": B
    }

def gen_mont_2ea_generators(A, B, prime_details):
    # Get the isomorphic short Weierstrass Model
    # https://eprint.iacr.org/2017/212.pdf, 2.4
    E2 = EllipticCurve([B^2*(1 - A^2/3), B^3 * (A/3) * ((2*A^2)/9 - 1)])
    while(True):
        # 2ea generators in Weierstrass Model coordinates
        w_P, w_Q = gen_2ea_generators(E2, prime_details, mont_gen=False)
        # Map generators back to Montgomery Model
        m_P = [w_P[0] / B - A/3, w_P[1] / (B^2)]
        m_Q = [w_Q[0] / B - A/3, w_Q[1] / (B^2)]

        # Some assertions to check if P, Q are indeed of the right order
        ea = prime_details["pow_2"]
        xP = m_P[0]
        xQ = m_Q[0]
        for j in range(ea - 1):
            xP = mont_xDBL(A, xP)
            xQ = mont_xDBL(A, xQ)
        assert(4*xP*(xP^2 + A*xP + 1) == 0)
        assert(4*xQ*(xQ^2 + A*xQ + 1) == 0) # The last xDBL(s) should be not possible, since we go to inf
        assert(xP != xQ)                    # They should also be linearly independent
    
        if (xP != 0 and xQ != 0):
            # We want one of the generator points to be (0 : 0 : 1) when multiplied by 2^ea
            continue
        else:
            if (xP == 0):
                return {
                    "xP": m_Q[0],
                    "yP": m_Q[1],
                    "xQ": m_P[0],
                    "yQ": m_P[1]
                }
            else:                
                return {
                    "xP": m_P[0],
                    "yP": m_P[1],
                    "xQ": m_Q[0],
                    "yQ": m_Q[1]
                }

def get_mont_point_diff(A, B, m_xP, m_yP, m_xQ, m_yQ):
    # Convert all to Weierstrass form, then do the computations there

    # Get the isomorphic short Weierstrass Model
    # https://eprint.iacr.org/2017/212.pdf, 2.4
    E2 = EllipticCurve([B^2*(1 - A^2/3), B^3 * (A/3) * ((2*A^2)/9 - 1)])
    P = E2(B*(m_xP + A/3), (B^2)*m_yP)
    Q = E2(B*(m_xQ + A/3), (B^2)*m_yQ)
    PQ = P - Q

    return {
        "xPQ": PQ[0] / B - A / 3,
        "yPQ": PQ[1] / (B^2)
    }

def gen_claw_instance(E, prime_details, P, Q, s=None):
    # (exponent of) Order of kernel point
    ea      = prime_details["pow_2"]
    ea_h    = ea / 2
    # Kernel point
    if (s is not None and not 0 <= s < 2**ea):
        raise ValueError(f"Secret {s} is not in range of [0, {2**ea})")
    elif (s is None):
        s = randrange(0, 2**ea)
    R = P + s*Q
    Rea_h = (2**int(ea_h)) * R
    # Generating codomains of isogenies
    #   By Sage functions
    mid_E_EA    = get_isog(E, Rea_h, 2)
    wEA         = get_isog(E, R, 2)
    wPA, wQA    = gen_2ea_generators(wEA, prime_details)
    #   By Montgomery formulas
    mEA         = get_mont_2_isog(E.a_invariants()[1], E.base_ring()(1), R[0], R[1], prime_details["pow_2"], R.base_ring())
    mEAGens     = gen_mont_2ea_generators(mEA["A"], mEA["B"], prime_details)
     # Check if the two codomains (Sage vs Mont formulas) are isomorphic
    assert(wEA.j_invariant() == 256*(mEA["A"]^2 -3)^3/(mEA["A"]^2 - 4))
    print("Codomain jinv:", wEA.j_invariant())

    # Get the differentials, P - Q and PA - QA
    PQ  = P - Q
    PQA = get_mont_point_diff(mEA["A"], mEA["B"], mEAGens["xP"], mEAGens["yP"], mEAGens["xQ"], mEAGens["yQ"])
    return {
        "xP": P[0],
        "yP": P[1],
        "xQ": Q[0],
        "yQ": Q[1],
        "xPQ":  PQ[0],
        "yPQ":  PQ[1],
        "E_A": E.a_invariants()[1],
        "E_B": E.base_ring()(1),
        "mid_jinv": mid_E_EA.j_invariant(),
        "xPA": mEAGens["xP"],
        "yPA": mEAGens["yP"],
        "xQA": mEAGens["xQ"],
        "yQA": mEAGens["yQ"],
        "xPQA": PQA["xPQ"],
        "yPQA": PQA["yPQ"],
        "EA_A": mEA["A"],
        "EA_B": mEA["B"],
    }

In [379]:
# Wrapping in a function to mute output, and for posterity
def gen_supersingular_curves():
    BITS = 64
    log3_2 = math.log(2, 3)
    for pow_2 in range(10, 32):
        for pow_3 in range(math.floor(log3_2 * pow_2) - 5, math.floor(log3_2 * pow_2) + 5):
            if is_prime(2^pow_2 * 3^pow_3 - 1):
                if 1/5 <= 2^pow_2 /  3^pow_3 <= 5/1:
                    p = 2^pow_2 * 3^pow_3 - 1
                    if check_supersingular(p):
                        print_prime_details(get_prime_details(pow_2, pow_3, 1, BITS))
                        print('')
    print_prime_details(get_prime_details(32, 20, 23, 128))

In [380]:
# Wrapping in a function to mute output, and for posterity
def verify_p434():
    lA, lB = 2, 3
    eA, eB = 216, 137 # p434 = 2^216*3^137-1
    p = lA ^ eA * lB ^ eB - 1
    assert p.is_prime()
    assert p % 4 == 3 # Necessary for below curve to be supersingular.
    #print("Is supersingular: ", check_supersingular(p))
    prime_details = get_prime_details(eA, eB, 1, 448)
    print_prime_details(prime_details)
    assert(p == from_hex_64chunks(to_hex_64chunks(p)))

    F2.<a> = GF(p^2, modulus=x^2 + 1)
    A = F2(6)
    B = F2(1)
    C = F2(1)
    S = B^1 * (1 - A^2/3)
    T = B^3 * A / 3 * (2*A^2/9 - 1)
    E = EllipticCurve(F2, [0, A, 0, 1, 0])
    print(E)

    m_xPA0 = from_hex_64chunks([0x05ADF455C5C345BF, 0x91935C5CC767AC2B, 0xAFE4E879951F0257, 0x70E792DC89FA27B1, 0xF797F526BB48C8CD, 0x2181DB6131AF621F, 0x00000A1C08B1ECC4])
    m_xPA1 = from_hex_64chunks([0x74840EB87CDA7788, 0x2971AA0ECF9F9D0B, 0xCB5732BDF41715D5, 0x8CD8E51F7AACFFAA, 0xA7F424730D7E419F, 0xD671EB919A179E8C, 0x0000FFA26C5A924A])

    m_xQA0 = from_hex_64chunks([0xFEC6E64588B7273B, 0xD2A626D74CBBF1C6, 0xF8F58F07A78098C7, 0xE23941F470841B03, 0x1B63EDA2045538DD, 0x735CFEB0FFD49215, 0x0001C4CB77542876])
    m_xQA1 = from_hex_64chunks([0xADB0F733C17FFDD6, 0x6AFFBD037DA0A050, 0x680EC43DB144E02F, 0x1E2E5D5FF524E374, 0xE2DDA115260E2995, 0xA6E4B552E2EDE508, 0x00018ECCDDF4B53E])

    m_xRA0 = from_hex_64chunks([0x01BA4DB518CD6C7D, 0x2CB0251FE3CC0611, 0x259B0C6949A9121B, 0x60E17AC16D2F82AD, 0x3AA41F1CE175D92D, 0x413FBE6A9B9BC4F3, 0x00022A81D8D55643])
    m_xRA1 = from_hex_64chunks([0xB8ADBC70FC82E54A, 0xEF9CDDB0D5FADDED, 0x5820C734C80096A0, 0x7799994BAA96E0E4, 0x044961599E379AF8, 0xDB2B94FBF09F27E2, 0x0000B87FC716C0C6])

    xPA0 = mont_red(prime_details, m_xPA0)
    xPA1 = mont_red(prime_details, m_xPA1)
    xPA = F2(xPA0 + xPA1 * a)
    yPA_roots = compute_y(E, xPA)
    #print(yPA_roots)

    PA1 = E(xPA, yPA_roots[0])
    PA2 = E(xPA, yPA_roots[1])

    xQA0 = mont_red(prime_details, m_xQA0)
    xQA1 = mont_red(prime_details, m_xQA1)
    xQA = F2(xQA0 + xQA1 * a)
    yQA_roots = compute_y(E, xQA)
    #print(yQA_roots)

    QA1 = E(xQA, yQA_roots[0])
    QA2 = E(xQA, yQA_roots[1])

    xRA0 = mont_red(prime_details, m_xRA0)
    xRA1 = mont_red(prime_details, m_xRA1)
    xRA = F2(xRA0 + xRA1 * a)

    print("2^eA:\t\t", 2^eA)
    print("Order PA:\t", PA1.order())
    print("Order QA:\t", QA1.order())
    print("Order PA + QA:\t", (PA1 + QA1).order())

    print("xRA:\t\t", xRA)
    print((PA1 - QA1)[0] == xRA)
    print((PA2 - QA1)[0] == xRA)
    print((PA1 - QA2)[0] == xRA)
    print((PA2 - QA2)[0] == xRA)

    m_xPB0 = from_hex_64chunks([0x6E5497556EDD48A3, 0x2A61B501546F1C05, 0xEB919446D049887D, 0x5864A4A69D450C4F, 0xB883F276A6490D2B, 0x22CC287022D5F5B9, 0x0001BED4772E551F])
    m_xPB1 = from_hex_64chunks([0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000])

    m_xQB0 = from_hex_64chunks([0xFAE2A3F93D8B6B8E, 0x494871F51700FE1C, 0xEF1A94228413C27C, 0x498FF4A4AF60BD62, 0xB00AD2A708267E8A, 0xF4328294E017837F, 0x000034080181D8AE])
    m_xQB1 = from_hex_64chunks([0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000])

    m_xRB0 = from_hex_64chunks([0x283B34FAFEFDC8E4, 0x9208F44977C3E647, 0x7DEAE962816F4E9A, 0x68A2BA8AA262EC9D, 0x8176F112EA43F45B, 0x02106D022634F504, 0x00007E8A50F02E37])
    m_xRB1 = from_hex_64chunks([0xB378B7C1DA22CCB1, 0x6D089C99AD1D9230, 0xEBE15711813E2369, 0x2B35A68239D48A53, 0x445F6FD138407C93, 0xBEF93B29A3F6B54B, 0x000173FA910377D3])

    xPB0 = mont_red(prime_details, m_xPB0)
    xPB1 = mont_red(prime_details, m_xPB1)
    xPB = F2(xPB0 + xPB1 * a)
    yPB_roots = compute_y(E, xPB)
    #print(yPB_roots)

    PB1 = E(xPB, yPB_roots[0])
    PB2 = E(xPB, yPB_roots[1])

    xQB0 = mont_red(prime_details, m_xQB0)
    xQB1 = mont_red(prime_details, m_xQB1)
    xQB = F2(xQB0 + xQB1 * a)
    yQB_roots = compute_y(E, xQB)
    #print(yQB_roots)

    QB1 = E(xQB, yQB_roots[0])
    QB2 = E(xQB, yQB_roots[1])

    xRB0 = mont_red(prime_details, m_xRB0)
    xRB1 = mont_red(prime_details, m_xRB1)
    xRB = F2(xRB0 + xRB1 * a)

    print("3^eB:\t\t", 3^eB)
    print("Order PB:\t", PB1.order())
    print("Order QB:\t", QB1.order())
    print("Order PB + QB:\t", (PB1 + QB1).order())

    print("xRB:\t\t", xRB)
    print((PB1 - QB1)[0] == xRB)
    print((PB2 - QB1)[0] == xRB)
    print((PB1 - QB2)[0] == xRB)
    print((PB2 - QB2)[0] == xRB)

In [381]:
lA, lB = 2, 3
eA, eB = 4, 3 # p63 = 2^22*3^15-1
p = lA ^ eA * lB ^ eB - 1
assert p.is_prime()
prime_details = get_prime_details(eA, eB, 1, 64)
print_prime_details(prime_details)
assert(p == from_hex_64chunks(to_hex_64chunks(p)))

F2.<a> = GF(p^2, modulus=x^2 + 1)
A = F2(6)
B = F2(1)
C = F2(1)
S = B^1 * (1 - A^2/3)
T = B^3 * A / 3 * (2*A^2/9 - 1)
E = EllipticCurve(F2, [0, A, 0, 1, 0])
print(E, E.is_supersingular())
print("j_invariant:", E.j_invariant())

P, Q = gen_2ea_generators(E, prime_details, mont_gen=True)
#print((2**(prime_details["pow_2"] - 1)) * P)
#print((2**(prime_details["pow_2"] - 1)) * Q)

pow_2 : {0x4}
pow_3 : {0x3}
cofactor : {0x1}
p : {0x1AF}
p_bits : 9
R : {0x0, 0x1}
R_bits : 64
p1 : {0x1B0}
px2 : {0x35E}
pinv_R : {0x90558804C0720AB1}
R2_p : {0xD8}
R_p : {0x151}
Elliptic Curve defined by y^2 = x^3 + 6*x^2 + x over Finite Field in a of size 431^2 True
j_invariant: 19


In [382]:
for s in range(0, 2**prime_details["pow_2"]):
    claw_instance = gen_claw_instance(E, prime_details, P, Q, s)
    print("Mid jinv:", claw_instance["mid_jinv"], '\n')
    # for key in claw_instance.keys():
    #     print(key, ":\t", claw_instance[key])

Codomain jinv: 143
Mid jinv: 189
Codomain jinv: 42*a + 141
Mid jinv: 209*a + 118
Codomain jinv: 364*a + 304
Mid jinv: 242
Codomain jinv: 389*a + 141
Mid jinv: 222*a + 118
Codomain jinv: 189
Mid jinv: 189
Codomain jinv: 125*a + 426
Mid jinv: 209*a + 118
Codomain jinv: 67
Mid jinv: 242
Codomain jinv: 381
Mid jinv: 222*a + 118
Codomain jinv: 189
Mid jinv: 189
Codomain jinv: 87*a + 190
Mid jinv: 209*a + 118
Codomain jinv: 67*a + 304
Mid jinv: 242
Codomain jinv: 344*a + 190
Mid jinv: 222*a + 118
Codomain jinv: 143
Mid jinv: 189
Codomain jinv: 381
Mid jinv: 209*a + 118
Codomain jinv: 234
Mid jinv: 242
Codomain jinv: 306*a + 426
Mid jinv: 222*a + 118
